# Exercício 1 - Titanic

Consideta o ficheiro train.csv, que contém o famoso dataset do Titanic, com informações de passageiros e se sobreviveram ou não. Eis um resumo completo:
| Coluna       | Tipo    | Descrição                                                         |
|--------------|---------|-------------------------------------------------------------------|
| PassengerId  | int64   | Identificador único do passageiro                                 |
| Survived     | int64   | Variável alvo: 1 = Sobreviveu, 0 = Não sobreviveu                 |
| Pclass       | int64   | Classe do bilhete (1ª, 2ª, 3ª)                                    |
| Name         | object  | Nome completo do passageiro                                       |
| Sex          | object  | Sexo (masculino/feminino)                                         |
| Age          | float64 | Idade                                                             |
| SibSp        | int64   | Nº de irmãos/cônjuges a bordo                                     |
| Parch        | int64   | Nº de pais/filhos a bordo                                         |
| Ticket       | object  | Número do bilhete                                                 |
| Fare         | float64 | Valor pago pelo bilhete                                           |
| Cabin        | object  | Número da cabine (muitos valores em falta)                        |
| Embarked     | object  | Porto de embarque: C = Cherbourg, Q = Queenstown, S = Southampton |



Este dataset é ideal para aplicar classificação supervisionada, pois a variável Survived é binária (0 ou 1).


No código seguinte fazemos:
1. Pré-processamento
1. Detecção de outliers
1. Codificação
1. Escalonamento
1. Treino e avaliação do modelo com Random Forest
1. Validação cruzada
1. Ajuste de hiperparâmetros com GridSearchCV
1. Testes adicionais: Curva ROC/AUC e comparação com Decision Tree

Modelos aplicados:
1. Random Forest Classifier
    * É o modelo principal que está a ser treinado.
    * É o modelo principal que está a ser treinado.
    * Avaliado com: classification_report() (Precision, Recall, F1-score); confusion_matrix(); Validação cruzada StratifiedKFold; GridSearchCV para otimizar hiperparâmetros; Se binário: cálculo da Curva ROC e AUC
1. Decision Tree Classifier
    * Serve como modelo de comparação.
    * Aplicado com profundidade máxima de 5.

# Classificação supervisionada

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.impute import SimpleImputer

import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# 1, carregar o dataset
df = pd.read_csv('data/titanic.csv')
print('Primeiras linhas do dataset:')
print(df.head())

Primeiras linhas do dataset:
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450

In [7]:
# 2. Exploração inicial
print('\nResumo estatístico:')
print(df.describe())

print('\nValores ausentes por coluna:')
print(df.isnull().sum())


Resumo estatístico:
       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   20.125000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200  

Valores ausentes por coluna:
PassengerId      0
Survived   

In [13]:
# 3. Tratamento de valores ausentes (numéricos com mediana, categóricos com moda)
num_cols = df.select_dtypes(include=np.number).columns
cat_cols = df.select_dtypes(exclude='object').columns

imputer_num = SimpleImputer(strategy='median')
df[num_cols] = imputer_num.fit_transform(df[num_cols])

imputer_cat = SimpleImputer(strategy='most_frequent')
df[cat_cols] = imputer_cat.fit_transform(df[cat_cols])

In [17]:
# 4. Detecção de outliers com Z-Score e remoção
from scipy.stats import zscore
z_scores = np.abs(zscore(df[num_cols]))

df = df[(z_scores < 3).all(axis=1)]
